---
# CS1F: Assessed exercise (Part 2)

This exercise is worth 8% of your mark in total for CS1F.
It should be completed individually.

This exercise has four stages, in consecutive CS1F lab sessions:

1. Designing a conceptual model of an interface.
2. Implementing a state machine.
3. Linking states to continuous control signals.
4. Experimental analysis of usability.

**You will require your results from previous weeks in each successive exercise!**

## A finite state machine
<img src="imgs/flashlight_design.png" width="60%">

In this lab you will have to implement, refine and adapt your initial designs from last week. 

This involves:

* Implement the state machine in Python.
* Test it, using a UI to simulate transitions.
* Adapt it as you see fit to make it work better.

Note that in this lab you will be testing only the state machine itself. In the following lab, you will look at how to interpret the motions from the touch slider, transforming continuous inputs into discrete state transitions. The purpose of this exercise is to explore how to analyse how an interface works *without* the specific details of how it is implemented.

Much of the code will be provided as a skeleton, but you will need to define the state machine functionality.

## Part 1: design

* Review your state machine design and update your diagrams to reflect changes to your design from last week that you want to make.
* **Draw the final, complete state diagram for your flashlight**. This should include all states and transitions.

## Part 2: implement

### Implementing the state machine

We will use a simple structure to define state machines. A state machine is represented by a Python **dictionary**, which maps pairs of `(state, action) -> new_state`. States and actions are just strings. For example, the Finite State Machine drawn below would be represented as this dictionary:

In [1]:
## IMPORTANT: you must specify the pair in the order (state, action)!


# The reason these are "off:off" or "on:on"
# is just that the
# part after the colon is used to set
# the flashlight state in the simulator.
#
# "off:off" is a state named "off", 
# which also turns "off" the flashlight when
# entered.
#
# See the end for a full list of flashlight
# states

on_off_fsm = {
        # (state, action)    -> state
          ("off:off", "turn_on") : "on:on",
          ("on:on",  "turn_off") : "off:off",
    }

## Simulating

The simulator below will take such a dictionary and let you simulate transitions interactively. It will show you the current state, the actions that are available as buttons on screen.

Before going further, try this. 


In [2]:
%load_ext autoreload
%autoreload 2

from fsm_simulator import fsm_simulate

# run the on off simulator specified above. 
# each state only has one action, 
# we specify the start state we will start in
results = fsm_simulate(machine=on_off_fsm, start_state="off:off")

# this line just hands over control to the window
# press ESC to exit
%gui xtk 

/anaconda3/lib/python3.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


### Timeouts
If an action is a *number* instead of a *string*, this will be interpreted as specifying a *timeout*. After this many seconds, the transition will happen, regardless of input.

In [3]:
toggle_fsm = {
        ("off:off", "turn_on") : "on:on",
        # after one second, return to the off
        # state, without any specific action required
        ("on:on", 1.0) : "off:off"
    }

In [4]:
 # run the toggle simulator specified above. 
results = fsm_simulate(machine=toggle_fsm, start_state="off:off")
%gui xtk 

A state can have many actions (transitions) leaving it, not just one:

In [6]:
# this FSM has both on/off actions *and* a toggle action
flip_off_fsm = {
        # state, action    -> state
        ("off:off", "turn_on") :   "on:on",
        ("off:off", "flip") :      "on:on",
        ("on:on",   "turn_off") :  "off:off",
        ("on:on",   "flip") :      "off:off",
    }

In [9]:
 # run the toggle simulator specified above. 
results = fsm_simulate(machine=flip_off_fsm, start_state="off:off")
%gui xtk 

## Results
`fsm_simulator` returns us the *history* of states visited during a simulation. These are recorded as a list of lists:
    
    [
        [time_enter, time_leave  from_state, action, to_state],
        [time_enter, time_leave, from_state, action, to_state],
        ...
    ]
    
We will use this in the last part of this lab (Part 4) to *analyse* how people interact with an interface. Below, the code just prints out spent in each state during the simulation, and the number of times each action was taken. You can try running the simulation again and checking you get different results. 

In [10]:
from fsm_report import summarise_report
summarise_report(results)

State dwell times
--------------------------------------------------------------------------------
off:off              7.8
on:on                0.9

Action counts
--------------------------------------------------------------------------------
START                1
flip                 4

State-action count table
--------------------------------------------------------------------------------
                 START      flip       
off:off              1          2      
on:on                0          2      


---

### Another report
We can try this with the on off state machine

In [11]:
results = fsm_simulate(on_off_fsm, start_state="off:off")
%gui xtk

In [12]:
summarise_report(results)

State dwell times
--------------------------------------------------------------------------------
off:off              4.9
on:on                3.2

Action counts
--------------------------------------------------------------------------------
START                1
turn_on              2
turn_off             2

State-action count table
--------------------------------------------------------------------------------
                 START      turn_on    turn_off   
off:off              1          2          0      
on:on                0          0          2      


## Fuzzing the FSM
If we run `fsm_simulator` with `fuzz=100` it will not ask for input and will not show a GUI. It will instead randomly choose 100 actions very quickly. This "fuzzing" operation lets us see where a randomly behaving user would end up -- do they get stuck? or do they move through states easily? This can be useful for debugging state machines. We can set `fuzz` to any number to run longer or shorter simulations.


In [13]:
 # run the on-off machine, but fuzz it instead
fuzz_results = fsm_simulate(machine=toggle_fsm, start_state="off:off", fuzz=100)
summarise_report(fuzz_results)

State dwell times
--------------------------------------------------------------------------------
off:off              0.0
on:on                0.0

Action counts
--------------------------------------------------------------------------------
START                1
turn_on              50
timeout:1.0          50

State-action count table
--------------------------------------------------------------------------------
                 START      turn_on    timeout:1. 
off:off              1          50         0      
on:on                0          0          50     


In this case, we can see that the on and off states are both visited equally often -- 50 times each -- which is what we'd expect since the only thing we can do is switch between them.

# Task

Implement the state machine that you designed in the last lab, with any refinements you have discussed in the cell below. Run it, both interactively *and* using the fuzzer. Print results out using `summarise_report()`. 


The flashlight simulator will set the flashlight any time a state with a name ending with `:<flashlight_state>` is entered. This is why the states were called "off:off" and "on:on". The state name was "off" and it also turned the flashlight off ":off". We could have used different names, as long as the colon part at the end is preserved:

In [14]:

a_b_fsm = {
        # (state, action)    -> state
        ("a:off", "turn_on") : "b:on",
        ("b:on",  "turn_off") : "a:off",
    }
fsm_simulate(machine=a_b_fsm, start_state="a:off")
%gui xtk

You should have states that cover these flashlight functions:

* `:off`
* `:locked`
* `:on`
* `:high`
* `:strobe`

You will almost certainly need some intermediate states as well to make a usable interface.

Questions:
* Can you reliably get to every output state (low beam, high beam, strobe, off, locked) from any other state?
* If you run with the fuzzer do you get to every state?
* Does the interface tend to get stuck in certain states?
* Can you find a way of modifying the state machine to be easier or quicker to use or explain? (imagine you had to explain how to use it to your parents, over Skype because they were stuck in the dark).
* Remember; you will have to execute the state transitions using a single slider, which might be clumsy to use. 
    * Think about how actions will be triggered. If you have five distinct actions, how would those be reliably sensed? If you only have one, how will you get to all of the states?


The example below is a very incomplete and very bad design:

In [15]:
## define your state machine by replacing this one (entirely)

# this is the worst possible design
# for a flashlight
flashlight_fsm = { 
    # (state, action) : new_state
    ("off:off", "press_on") : "wait",
    ("wait", 1.5) : "mad:strobe",
    ("mad:strobe", "press_off"): "wait:off",
    ("wait:off", 0.5): "wait"
}

In [16]:
fuzz_results = fsm_simulate(machine=flashlight_fsm, start_state="off:off", fuzz=100)

	off:off


In [17]:
print("Fuzz testing")
summarise_report(fuzz_results)

Fuzz testing
State dwell times
--------------------------------------------------------------------------------
off:off              0.0
wait                 0.0
mad:strobe           0.0
wait:off             0.0

Action counts
--------------------------------------------------------------------------------
START                1
press_on             1
timeout:1.5          33
press_off            33
timeout:0.5          33

State-action count table
--------------------------------------------------------------------------------
                 START      press_on   timeout:1. press_off  timeout:0. 
off:off              1          1          0          0          0      
wait                 0          0          33         0          0      
mad:strobe           0          0          0          33         0      
wait:off             0          0          0          0          33     


In [21]:
results = fsm_simulate(machine=flashlight_fsm, start_state="off:off")
%gui xtk

	off:off


In [22]:
print("Manual testing")
summarise_report(results)

Manual testing
State dwell times
--------------------------------------------------------------------------------
off:off              6.5
wait                 7.5
mad:strobe           6.5
wait:off             2.0

Action counts
--------------------------------------------------------------------------------
START                1
press_on             1
timeout:1.50         5
press_off            4
timeout:0.50         4

State-action count table
--------------------------------------------------------------------------------
                 START      press_on   timeout:1. press_off  timeout:0. 
off:off              1          1          0          0          0      
wait                 0          0          5          0          0      
mad:strobe           0          0          0          4          0      
wait:off             0          0          0          0          4      
